Problem Statement: Provide suggestion for medical check-up booths amid COVID-19 based on the areas, population,
                   total number of cases in that area, hospitals in that area(to check the current medical 
                   facility status etc)
        
Target Audience: Government, medical agencies, NGOs and all related to COVID-19
    
Observed Problem: During the prime time of COVID-19, there was a requirement to identify the places where the                         medical and food supply is less and cases are more. Due to panic situation, a lot number of small                   industries are shut down and due to which there is lack of medicine and food. 
                  So I planned to identify the areas district wise in Delhi where proper supply of medicine and                       food is not there.
        

Approach to solve this problem: 
1. Call the FourSquare API to get the details of Hospitals in Delhi
2. Collect the information about the hospitals with proper address,Pincode, sub-districts etc
3. Collect the information about each districts such as population, area, density etc.
4. Lastly collect the information about the current COVID-19 cases in each districts.


Next steps: 
 1. We will combine the data which we have collected district wise.
 2. Then we will analyse the data based on parameters such as population, density, current cases etc.
 3. Then from FourSquare we will try to get the information about the hospitals such as ratings etc.
 4. Lastly we will make clusters where we need to provide the medicines and other basic amenities. 

In [77]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from sklearn.cluster import KMeans
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
import folium
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import urllib.request
!conda install beautifulsoup4 --yes
!conda install lxml --yes
print('lxml installed')
from bs4 import BeautifulSoup
print("Libraries imported")

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be UPDATED:

  ca-certificates     pkgs/main::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2020.4.5.1-hecc5488_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi              pkgs/main::certifi-2020.4.5.1-py36_0 --> conda-forge::certifi-2020.4.5.1-py36h9f0ad1d_0
  openssl              pkgs/main::openssl-1.1.1g-h7b6447c_0 --> conda-forge::openssl-1.1.1g-h516909a_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.4.5~ --> pkgs/main::ca-certificates-2020.

In [78]:
#This is our first dataset which we have collected.
df1_hosp=pd.read_csv('hospitals_district.csv')

#Removing the unnecessary column
df1_hosp = df1_hosp.drop(columns='Unnamed: 0',axis=1)
df1_hosp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Names          50 non-null     object 
 1   Sub Districts  50 non-null     object 
 2   Districts      50 non-null     object 
 3   Latidude       50 non-null     float64
 4   Longitude      50 non-null     float64
 5   State          50 non-null     object 
 6   Postal Code    50 non-null     int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 2.9+ KB


In [80]:
#make groups by Districts
df1_hosp.groupby(by='Districts', axis=0).first()

,Names,Sub Districts,Latidude,Longitude,State,Postal Code
Districts,,,,,,
Central Delhi,Sir Gangaram Hospital,Rajender Nagar,28.638601,77.188884,Delhi,110060
New Delhi,Dr. Ram Manohar Lohia Hospital,Connaught Place,28.624654,77.200609,Delhi,110001
North Delhi,Tirath Ram Shah Hospital,Tis Hazari,28.670918,77.217385,Delhi,110054
North West Delhi,Karma Ayurveda Hospital,Pitam Pura,28.598992,77.189941,Delhi,110034
South Delhi,G.M. Modi Hospital,Saket,28.528471,77.215077,Delhi,110017
South West Delhi,army hospital research and referal,Dhaula Kuan,28.586160,77.157681,Delhi,110010
West Delhi,Orchid Hospital,Janakpuri,28.612374,77.086032,Delhi,110058


In [81]:
#check for the null values
df1_hosp.isnull().sum()

Names            0
Sub Districts    0
Districts        0
Latidude         0
Longitude        0
State            0
Postal Code      0
dtype: int64

In [82]:
#create a folium map for above data
districts_map = folium.Map(location=[latitude, longitude], zoom_start=11) 

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df1_hosp['Latidude'], df1_hosp['Longitude'],df1_hosp['Districts']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(districts_map)

# display map
districts_map

In [83]:
#This is our second data which contains population,Areas, density etc.
# Get the delhi data with districts
delhi_dis='https://www.indianmirror.com/india-post/indianpincode.html'

#Using web scraping we fetched the table
df2_pop=pd.read_html(delhi_dis, header=0)[6]

df2_pop.head()

,Code,District,Headquarters,Pincode,Area code,Latitude & Longitude,Population (2001),Area (/km²),Density (/km²)
0,CD,Central Delhi,Darya Ganj,110002,NaN,28° 51' N & 77° 25' E,644005,25,25759
1,ED,East Delhi,Preet Vihar,110003,NaN,28° 37' N & 77° 13' E,1448770,440,3293
2,ND,New Delhi,Connaught Place,110001,11.0,28° 36' N & 77° 12' E,302363,22,13744
3,NO,North Delhi,Sadar Bazar,110007,NaN,28° 63' N & 77° 22' E,779788,59,12996
4,NE,North East Delhi,Shahdara,110032,NaN,28° 4' N & 77° 2' E,1763712,52,33917


In [84]:
#lets remove the columns which are not required
df2_pop = df2_pop.drop(columns=['Code','Area code','Latitude & Longitude'])

#Make the column name of Districts uniform for all the tables
df2_pop = df2_pop.rename(columns={'District':'Districts'})
df2_pop

,Districts,Headquarters,Pincode,Population (2001),Area (/km²),Density (/km²)
0,Central Delhi,Darya Ganj,110002,644005,25,25759
1,East Delhi,Preet Vihar,110003,1448770,440,3293
2,New Delhi,Connaught Place,110001,302363,22,13744
3,North Delhi,Sadar Bazar,110007,779788,59,12996
4,North East Delhi,Shahdara,110032,1763712,52,33917
5,North West Delhi,Kanjhawala,110081,785284,130,6041
6,South Delhi,Saket,110017,2258367,250,9033
7,South West Delhi,Vasant Vihar,110057,1749492,395,4430
8,West Delhi,Rajouri Garden,110027,2119641,112,18925


In [85]:
#Now we will merge above 2 dataframes to get populations of the districts
#merge on the basis of Districts
df_comb = pd.merge(df1_hosp,df2_pop,on="Districts")
df_comb.head()

,Names,Sub Districts,Districts,Latidude,Longitude,State,Postal Code,Headquarters,Pincode,Population (2001),Area (/km²),Density (/km²)
0,Dr. Ram Manohar Lohia Hospital,Connaught Place,New Delhi,28.624654,77.200609,Delhi,110001,Connaught Place,110001,302363,22,13744
1,Sucheta Kriplani Hospital,Connaught Place,New Delhi,28.634001,77.221702,Delhi,110001,Connaught Place,110001,302363,22,13744
2,Rml hospital,Connaught Place,New Delhi,28.636175,77.202007,Delhi,110001,Connaught Place,110001,302363,22,13744
3,"Sucheta Kripalini hospital , new delhi",Connaught Place,New Delhi,28.638733,77.212539,Delhi,110001,Connaught Place,110001,302363,22,13744
4,ICARE Eye Hospital,Hauz Khas,New Delhi,28.579491,77.335896,Delhi,110016,Connaught Place,110001,302363,22,13744


In [91]:
#Below is our third data source which contains the active cases of COVID-19
#Now we will work with our last dataset
cases_data='https://api.covid19india.org/csv/latest/district_wise.csv'

#This will give us the list of Districts from all India. We need to filter them in next steps
df_cases=pd.read_csv(cases_data)

df_cases.head(5)

,SlNo,State_Code,State,District_Key,District,Confirmed,Active,Recovered,Deceased,Delta_Confirmed,Delta_Active,Delta_Recovered,Delta_Deceased,District_Notes,Last_Updated
0,1,AN,Andaman and Nicobar Islands,AN_Nicobars,Nicobars,0,0,0,0,0,0,0,0,NaN,NaN
1,2,AN,Andaman and Nicobar Islands,AN_North and Middle Andaman,North and Middle Andaman,1,0,1,0,0,0,0,0,NaN,NaN
2,3,AN,Andaman and Nicobar Islands,AN_South Andaman,South Andaman,32,0,32,0,0,0,0,0,NaN,NaN
3,4,AP,Andhra Pradesh,AP_Anantapur,Anantapur,115,63,48,4,0,-3,3,0,NaN,NaN
4,5,AP,Andhra Pradesh,AP_Chittoor,Chittoor,131,57,74,0,10,10,0,0,NaN,NaN


In [92]:
#Filtering the dataset from all India Districts to the districts available in State Delhi
df_cases = df_cases[df_cases['State']== 'Delhi']
df_cases.head()

,SlNo,State_Code,State,District_Key,District,Confirmed,Active,Recovered,Deceased,Delta_Confirmed,Delta_Active,Delta_Recovered,Delta_Deceased,District_Notes,Last_Updated
142,144,DL,Delhi,DL_Central Delhi,Central Delhi,184,184,0,0,0,0,0,0,NaN,NaN
143,145,DL,Delhi,DL_East Delhi,East Delhi,38,38,0,0,0,0,0,0,NaN,NaN
144,146,DL,Delhi,DL_New Delhi,New Delhi,37,37,0,0,0,0,0,0,NaN,NaN
145,147,DL,Delhi,DL_North Delhi,North Delhi,60,60,0,0,0,0,0,0,NaN,NaN
146,148,DL,Delhi,DL_North East Delhi,North East Delhi,25,25,0,0,0,0,0,0,NaN,NaN


In [93]:
#As we can see that not all the columns are of much use for us. For example Recovered,Deceased are not of much use
#here for our analysis
#So we will take only the important columns as below
df_cases = df_cases[['State','District','Confirmed']].reset_index(drop=True)

#Similarly we are making name of column District uniform for all the tables
df_cases = df_cases.rename(columns={'District':'Districts'})
df_cases.head()

,State,Districts,Confirmed
0,Delhi,Central Delhi,184
1,Delhi,East Delhi,38
2,Delhi,New Delhi,37
3,Delhi,North Delhi,60
4,Delhi,North East Delhi,25


In [95]:
#Now we will merge the above dataset with the dataset which contains population and Hospitals Address 
df_final = pd.merge(df_comb,df_cases,on='Districts')

#Here we are dropping the duplicate columns as Pincode is available in multiple tables
df_final=df_final.drop(columns=['Headquarters','Pincode','State_y'])
df_final.head()

,Names,Sub Districts,Districts,Latidude,Longitude,State_x,Postal Code,Population (2001),Area (/km²),Density (/km²),Confirmed
0,Dr. Ram Manohar Lohia Hospital,Connaught Place,New Delhi,28.624654,77.200609,Delhi,110001,302363,22,13744,37
1,Sucheta Kriplani Hospital,Connaught Place,New Delhi,28.634001,77.221702,Delhi,110001,302363,22,13744,37
2,Rml hospital,Connaught Place,New Delhi,28.636175,77.202007,Delhi,110001,302363,22,13744,37
3,"Sucheta Kripalini hospital , new delhi",Connaught Place,New Delhi,28.638733,77.212539,Delhi,110001,302363,22,13744,37
4,ICARE Eye Hospital,Hauz Khas,New Delhi,28.579491,77.335896,Delhi,110016,302363,22,13744,37


Above is our final data which is available for analysis. 
Now we will start fetching the details of the Delhi from FourSquare API which is our main task.

In [112]:
#Our first step to consume our FourSquare API
CLIENT_ID = 'XXXXXXXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXXXXXX' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100

In [97]:
#Providing the location as New Delhi
address = 'New Delhi'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

28.6141793 77.2022662


In [98]:
#We are providing search query as "Hospital" as we want information about the Hospitals in Delhi
#Providing radius as 100 Kms or 100000 metres
search_query = 'Hospital'
radius = 100000
print(search_query + ' .... OK!')

Hospital .... OK!


In [99]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PHTW51CXQQ4E41FAF3AJZMW2MSVCCL1GN2FWIL2OV43OAVC4&client_secret=32EO1EG3OPPHROKBAVXVPNYLBOD5VD5GZEWX2MBTFC1FQ1RU&ll=28.6141793,77.2022662&v=20180604&query=Hospital&radius=100000&limit=100'

In [100]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ebaf34647e0d6001b1b414a'},
 'response': {'venues': [{'id': '4e1f1430cc3f012b9756fdfd',
    'name': 'Dr. Ram Manohar Lohia Hospital',
    'location': {'address': 'Ram Manohar Lohia Hospital',
     'lat': 28.624654222133696,
     'lng': 77.20060862393855,
     'labeledLatLngs': [{'label': 'display',
       'lat': 28.624654222133696,
       'lng': 77.20060862393855}],
     'distance': 1177,
     'cc': 'IN',
     'city': 'New Delhi',
     'state': 'Delhi',
     'country': 'India',
     'formattedAddress': ['Ram Manohar Lohia Hospital',
      'New Delhi',
      'Delhi',
      'India']},
    'categories': [{'id': '4bf58dd8d48988d196941735',
      'name': 'Hospital',
      'pluralName': 'Hospitals',
      'shortName': 'Hospital',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1589310363',
    'hasPerk': False},
   {'id': '4f3a7e46e4b03d32dfbeb80a'

In [102]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df_v2 = json_normalize(venues)
df_v2.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode,location.neighborhood
0,4e1f1430cc3f012b9756fdfd,Dr. Ram Manohar Lohia Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1589310363,False,Ram Manohar Lohia Hospital,28.624654,77.200609,"[{'label': 'display', 'lat': 28.62465422213369...",1177,IN,New Delhi,Delhi,India,"[Ram Manohar Lohia Hospital, New Delhi, Delhi,...",NaN,NaN,NaN
1,4f3a7e46e4b03d32dfbeb80a,Sucheta Kriplani Hospital Cafeteria,"[{'id': '4bf58dd8d48988d1c7941735', 'name': 'S...",v-1589310363,False,NaN,28.634001,77.221702,"[{'label': 'display', 'lat': 28.63400068219633...",2911,IN,NaN,NaN,India,[India],NaN,NaN,NaN
2,4be7f5fe947820a1dc44b4db,Sir Gangaram Hospital | सर गंगाराम अस्पताल,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1589310363,False,Rajendra Nagar,28.638601,77.188884,"[{'label': 'display', 'lat': 28.6386009566119,...",3016,IN,Delhi,Delhi,India,"[Rajendra Nagar (Karol Bagh), Delhi, Delhi, In...",Karol Bagh,NaN,NaN
3,4d849f985ad3a09319ecd2fd,army hospital research and referal,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1589310363,False,near vasant vihar,28.586160,77.157681,"[{'label': 'display', 'lat': 28.58616016121680...",5358,IN,Delhi,Delhi,India,"[near vasant vihar (Dhaula Kuan), Delhi, Delhi...",Dhaula Kuan,NaN,NaN
4,4db647200437fa536a53e602,St. Stephen's Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1589310363,False,Tis Hazari,28.666454,77.214489,"[{'label': 'display', 'lat': 28.6664535219038,...",5940,IN,New Delhi,Delhi,India,"[Tis Hazari, New Delhi, Delhi, India]",NaN,NaN,NaN


In [103]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_v2.columns if col.startswith('location.')] + ['id']
dataframe_filtered = df_v2.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,neighborhood,id
0,Dr. Ram Manohar Lohia Hospital,Hospital,Ram Manohar Lohia Hospital,28.624654,77.200609,"[{'label': 'display', 'lat': 28.62465422213369...",1177,IN,New Delhi,Delhi,India,"[Ram Manohar Lohia Hospital, New Delhi, Delhi,...",NaN,NaN,NaN,4e1f1430cc3f012b9756fdfd
1,Sucheta Kriplani Hospital Cafeteria,Snack Place,NaN,28.634001,77.221702,"[{'label': 'display', 'lat': 28.63400068219633...",2911,IN,NaN,NaN,India,[India],NaN,NaN,NaN,4f3a7e46e4b03d32dfbeb80a
2,Sir Gangaram Hospital | सर गंगाराम अस्पताल,Hospital,Rajendra Nagar,28.638601,77.188884,"[{'label': 'display', 'lat': 28.6386009566119,...",3016,IN,Delhi,Delhi,India,"[Rajendra Nagar (Karol Bagh), Delhi, Delhi, In...",Karol Bagh,NaN,NaN,4be7f5fe947820a1dc44b4db
3,army hospital research and referal,Hospital,near vasant vihar,28.586160,77.157681,"[{'label': 'display', 'lat': 28.58616016121680...",5358,IN,Delhi,Delhi,India,"[near vasant vihar (Dhaula Kuan), Delhi, Delhi...",Dhaula Kuan,NaN,NaN,4d849f985ad3a09319ecd2fd
4,St. Stephen's Hospital,Hospital,Tis Hazari,28.666454,77.214489,"[{'label': 'display', 'lat': 28.6664535219038,...",5940,IN,New Delhi,Delhi,India,"[Tis Hazari, New Delhi, Delhi, India]",NaN,NaN,NaN,4db647200437fa536a53e602


In [104]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) 

# add a red circle marker to represent the New Delhi
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hospitals',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Hospitals as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [105]:
#Now we are trying to fetch the data of a particular hospital using ID
venue_id='4c0b6d8f6071a5930644e132'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4c0b6d8f6071a5930644e132?client_id=PHTW51CXQQ4E41FAF3AJZMW2MSVCCL1GN2FWIL2OV43OAVC4&client_secret=32EO1EG3OPPHROKBAVXVPNYLBOD5VD5GZEWX2MBTFC1FQ1RU&v=20180604'

In [106]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4c0b6d8f6071a5930644e132',
 'name': 'TDI Mall',
 'contact': {},
 'location': {'address': 'Rajouri garden',
  'crossStreet': 'Near Vishal Cinems',
  'lat': 28.65091714108544,
  'lng': 77.12072021895139,
  'labeledLatLngs': [{'label': 'display',
    'lat': 28.65091714108544,
    'lng': 77.12072021895139}],
  'cc': 'IN',
  'city': 'New Delhi',
  'state': 'Delhi',
  'country': 'India',
  'formattedAddress': ['Rajouri garden (Near Vishal Cinems)',
   'New Delhi',
   'Delhi',
   'India']},
 'canonicalUrl': 'https://foursquare.com/v/tdi-mall/4c0b6d8f6071a5930644e132',
 'categories': [{'id': '4bf58dd8d48988d1fd941735',
   'name': 'Shopping Mall',
   'pluralName': 'Shopping Malls',
   'shortName': 'Mall',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 8},
 'likes': {'count': 6,
  'groups': [{'type': 'others',
    'count': 6,
    'items': [{'id': '56647181',
      'firstNa

In [107]:
# Using below code to get the rating of the hospital
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.2


In [109]:
venue_id = '4c0b6d8f6071a5930644e132' # ID of Hospital
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.2


In [110]:
venue_id = '5c5b7677c9f907002b6fe60d' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


So here all our data collection is complete.
This completes our first week Problem Statement and data collection.
Motivation behind doing this.